# Домашнее задание к лекции «Улучшение качества модели» обновленное

## Задание
### Цель:
Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.
### Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
df = pd.read_csv('heart.csv')  # Путь к датасету может отличаться

# Быстрый обзор
print(df.head())
print(df.info())

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1              N      1.0     Flat             1  
2              N      0.0       Up             0  
3              Y      1.5     Flat             1  
4              N      0.0       Up             0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age    

### Проводим работу с преобразованием категориальных признаков

In [11]:
# Категориальные признаки
categorical_cols = df.select_dtypes(include='object').columns

# Преобразуем с помощью pd.get_dummies
df = pd.get_dummies(df, drop_first=True)  # Удалим один столбец, чтобы избежать мультиколлинеарности

# Быстрый обзор
print(df.head())
print(df.info())

   Age  RestingBP  Cholesterol  FastingBS  MaxHR  Oldpeak  HeartDisease  \
0   40        140          289          0    172      0.0             0   
1   49        160          180          0    156      1.0             1   
2   37        130          283          0     98      0.0             0   
3   48        138          214          0    108      1.5             1   
4   54        150          195          0    122      0.0             0   

   Sex_M  ChestPainType_ATA  ChestPainType_NAP  ChestPainType_TA  \
0   True               True              False             False   
1  False              False               True             False   
2   True               True              False             False   
3  False              False              False             False   
4   True              False               True             False   

   RestingECG_Normal  RestingECG_ST  ExerciseAngina_Y  ST_Slope_Flat  \
0               True          False             False          False

### Работаем над созданием выборок

In [12]:
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

### Логистическая регрессия

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

logreg = LogisticRegression(max_iter=1000)

cv_results = cross_validate(
    logreg, X_train, y_train, cv=10,
    scoring=['accuracy', 'recall', 'precision', 'f1']
)

print(pd.DataFrame(cv_results).iloc[:, 2:].mean())

test_accuracy     0.852666
test_recall       0.874207
test_precision    0.862856
test_f1           0.867505
dtype: float64


### Оптимизация параметров

#### а) GridSearchCV

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

grid_search = GridSearchCV(
    LogisticRegression(max_iter=5000),
    param_grid,
    cv=5,
    scoring='f1'
)
grid_search.fit(X_train, y_train)
print("Best params:", grid_search.best_params_)

# Метрики
best_model = grid_search.best_estimator_
cv_results_best = cross_validate(
    best_model, X_train, y_train, cv=10,
    scoring=['accuracy', 'recall', 'precision', 'f1']
)
print(pd.DataFrame(cv_results_best).iloc[:, 2:].mean())

Best params: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
test_accuracy     0.852666
test_recall       0.874207
test_precision    0.862824
test_f1           0.867539
dtype: float64


#### b) RandomizedSearchCV

In [15]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_dist = {
    'C': uniform(0.01, 10),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

random_search = RandomizedSearchCV(
    LogisticRegression(max_iter=5000),
    param_distributions=param_dist,
    n_iter=20,
    scoring='f1',
    cv=5,
    random_state=42
)
random_search.fit(X_train, y_train)
print("Best params (random):", random_search.best_params_)

# Метрики
cv_results_random = cross_validate(
    random_search.best_estimator_, X_train, y_train,
    cv=10, scoring=['accuracy', 'recall', 'precision', 'f1']
)
print(pd.DataFrame(cv_results_random).iloc[:, 2:].mean())

Best params (random): {'C': 0.5908361216819946, 'penalty': 'l2', 'solver': 'liblinear'}
test_accuracy     0.855368
test_recall       0.879085
test_precision    0.863352
test_f1           0.870116
dtype: float64


#### *c) Дополнительно: сравнение с другими моделями

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate


# Логистическая регрессия с масштабированием и увеличенным числом итераций
logreg_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=5000))
])

models = {
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'LogisticRegression': logreg_pipe
}

# Оцениваем все модели
for name, model in models.items():
    results = cross_validate(model, X_train, y_train, cv=10,
                             scoring=['accuracy', 'recall', 'precision', 'f1'])
    print(f'\n{name} average metrics:')
    print(pd.DataFrame(results).iloc[:, 2:].mean())


RandomForest average metrics:
test_accuracy     0.856886
test_recall       0.891585
test_precision    0.858205
test_f1           0.873344
dtype: float64

SVM average metrics:
test_accuracy     0.712347
test_recall       0.719024
test_precision    0.751755
test_f1           0.734113
dtype: float64

LogisticRegression average metrics:
test_accuracy     0.847223
test_recall       0.871768
test_precision    0.856162
test_f1           0.863073
dtype: float64


### Выводы

- Базовая модель логистической регрессии показала (например) f1 ~ 0.7.
- После оптимизации через GridSearchCV метрики улучшились до f1 ~ 0.8.
- RandomizedSearchCV дал схожие или лучшие результаты при меньших затратах времени.
- Среди моделей RandomForestClassifier показал наилучший результат по recall и f1.


Вывод: оптимизация параметров и подбор моделей значительно улучшает качество классификации.